<img src="./assets/mellea_banner.png" width="80%"/>

# Mellea workshop

[Notebook on github](https://github.com/remiserra/mellea-workshop/blob/main/mellea_workshop.ipynb),
[Notebook on colab](https://colab.research.google.com/github/remiserra/mellea-workshop/blob/main/mellea_workshop.ipynb)

This workshop will introduce the basics of Generative Computing through a series of labs following a use-case.

During this workshop, we will:
1. Get up an running with Mellea.
2. See the Instruct - Validate - Repair pattern in action.
3. Encapsulate LLM calls using a functional interface via Mellea's `@generative` decorator.
4. Combine Mellea and Docling to write a generative program that operates over documents.
5. Leverage a vision model to analyse an image.

# Warmup

Run the first cell during our introduction. The first cell will:
 * download an install ollama on your Colab instance
 * pull the granite4 and granite-vision model weights from ollama
 * pull the docling model weights

This should take about 10 minutes.

In [21]:
print("Install ollama")
# - When running locally, install ollama app from https://ollama.com, and the service should start automatically with mac/windows
# - When running on colab uncomment the next two lines to install and start ollama in the colab environment:
# !curl -fsSL https://ollama.com/install.sh | sh > /dev/null
# !nohup ollama serve >/dev/null 2>&1 &

print("Download model weights")
# Download model weights for the Granite 4 and Granite vision models
!ollama pull ibm/granite4:micro
!ollama pull granite3.2-vision
print("Install Mellea with Docling")
!uv pip install mellea docling jupyter ipywidgets 
print("Run docling once to download model weights")
from mellea.stdlib.docs.richdocument import RichDocument
rd = RichDocument.from_document_file("https://arxiv.org/pdf/1906.04043")

# Some UI niceness
# from IPython.display import HTML, display  # noqa: E402

# def set_css():
#     display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))

# get_ipython().events.register("pre_run_cell", set_css)

print("Ready!")

Install ollama
Download model weights
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 6c02683809a8: 100% ▕██████████████████▏ 2.1 GB                         
pulling 0f6ec9740c76: 100% ▕██████████████████▏ 7.1 KB                         
pulling cfc7749b96f6: 100% ▕██████████████████▏  11 KB                         
pulling ba32b08db168: 100% ▕██████████████████▏  417 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 1aefcd9a8a15: 100% ▕██████████████████▏ 1.5 GB                         
pulling 4d464be24899: 100% ▕██████████████████▏ 892 MB                         
pulling 579046ba1157: 100% ▕██████████████████▏ 1.3 KB                         
pulling 1fa69e2371b7: 100% ▕██████████████████▏  154 B             

2025-10-23 11:27:28,076 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-23 11:27:28,081 - INFO - Going to convert document batch...
2025-10-23 11:27:28,082 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 77ade624737fd5e44f3478e1fd0b64f8
2025-10-23 11:27:28,083 - INFO - Auto OCR model selected ocrmac.
2025-10-23 11:27:28,084 - INFO - Accelerator device: 'mps'
2025-10-23 11:27:32,079 - INFO - Accelerator device: 'mps'
2025-10-23 11:27:32,509 - INFO - Processing document 1906.04043v1.pdf
2025-10-23 11:27:37,979 - INFO - Finished converting document 1906.04043v1.pdf in 10.18 sec.


Ready!


# Introduction to Mellea.ai

## *“The second most remarkable thing about LLMs is that they can understand and produce fluent natural language. The most remarkable thing about them is that they do the wrong thing 5-50% of the time”*

### > **We need structures** that make it a first-class concern **to check our work**, and ideally, we should build new tools to allow interrogation of certainty.

<hr/>

## What is “generative computing”?

*Generative computing* is the idea that **LLMs** can function **as computing elements** that are a part of, not separate from, the rest of computer science.

Like computing in general, this will have many facets, spanning engineering practice, system design, theory, hardware, software, etc.

Many aspects are already emerging in the field, but they could use a nudge, and **we’re building tools to accelerate progress**.

<hr/>

## Mellea.ai

### Local-first Capability
Generative programs can do Big Model Things without Big Model hardware.

### Robust and Composable
Requirement-driven inference pipelines result in rock-solid libraries and deployable apps.

### Ready to Scale
Swap out inference engines and models with ease. Massively scale inference in one line of code.

### Open & Permissive
The open inference stack for building generative programs on open weight models.

<hr/>

# Workshop: Applied Principles for Mellea.ai

5 steps to showcase Mellea features with a unique workshop:

### [01. Easy prompting](#lab-1-hello-mellea)
Automated response via LLM

### [02. Validate & Retry](#lab-2-instruct-validate-repair)
Generated response guided-verification

### [03. Function-Generation](#lab-3-generative-stubs)
Satisfaction Detection through Stub Generation

### [04. Embedded Document processing](#lab-4-docling-and-mellea)
Invoice simplified Analysis

### [05. Vision Model](#lab-5-vision-models)
Product identification in a photo

<hr/>

## Use-case overview

Let's sell T-Shirts on our [Mellea.ai Store](https://mellea-ai-store.figma.site/)!

<iframe src="https://mellea-ai-store.figma.site/" width="100%" height=400 />

<hr/>

# Lab 1: Hello, Mellea!

## Mellea features

- Run locally with Ollama
- Get started with 3 lines of code

## Use-case illustration

<TBD>

## Lab guidelines

Running `mellea.start_session()` initialize a new `MelleaSession`. The session holds three things:
1. The model to use for this session. In this tutorial we will use granite3.3:8b.
2. An inference engine; i.e., the code that actually calls our model. We will be using ollama, but you can also use Huggingface or any OpenAI-compatible endpoint.
3. A `Context`, which tells Mellea how to remember context between requests. This is sometimes called the "Message History" in other frameworks. Throughout this tutorial, we will be using a `SimpleContext`. In `SimpleContext`s, **every request starts with a fresh context**. There is no preserved chat history between requests. Mellea provides other types of context, but today we will not be using those features. See the Tutorial for further details.

## Let's experiment!

In [22]:
import mellea

m = mellea.start_session()

answer = m.chat("tell me about IBM and the early history of AI.")

print(answer.content)

2025-10-23 11:27:38,008 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/ps "HTTP/1.1 200 OK"
2025-10-23 11:27:38,019 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
2025-10-23 11:27:45,459 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


IBM (International Business Machines Corporation) has been at the forefront of artificial intelligence (AI) since its inception in 1953.

The term "artificial intelligence" was first coined by John McCarthy, an American computer scientist, when he attended a conference at Dartmouth College in 1956. IBM played a crucial role from the very beginning as it is one of the pioneers in this field.

In 1959, IBM developed SNLA (Symbolic Natural Language Analysis), which was designed to perform natural language processing tasks. This marked IBM's first significant step into AI research and development.

A major milestone came when Arthur Samuel, an IBM researcher, created a checkers-playing program that could learn from its own experiences. This demonstrated the potential of machine learning - a subset of AI where systems can analyze data and act upon it without being explicitly programmed to do so.

In 1961, IBM developed the first prototype for what would become their renowned Expert System, 

## Solution

```py
import mellea
m = mellea.start_session()
answer = m.chat("tell me about IBM and the early history of AI.")
print(answer.content)
```

<hr/>

# Lab 2: Instruct-Validate-Repair

## Mellea features

Instruct-Validate-Repair is a design pattern for building robust automation using LLMs. The idea is simple:
1. Instruct the model to perform a task and specify requirements on the output of the task.
2. Validate that these requirements are satisfied by the model's output.
3. If any requirements fail, try to repair.

## Use-case illustration

We will prepare an e-mail answer, and check for specific aspects in the generated email.

## Lab guidelines

- Build a requirements list with the `req()` and `check()` constructors.
- use the `instruct()` method to ask Mellea to perform an instruction and retry until the requirements are met.

## Let's experiment!

In [23]:
import mellea
from mellea.stdlib.requirement import check, req, simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

m = mellea.start_session()

requirements_list = [
    req("The email should have a salutation"),
    req(
        "Use only lower-case letters",
        validation_fn=simple_validate(lambda x: x.lower() == x),
    ),
    check("Do not mention purple elephants."),
]

name = "Olivia"
notes = "Olivia helped the lab over the last few weeks by organizing intern events, advertising the speaker series, and handling issues with snack delivery."

email_candidate = m.instruct(
    description="Write an email to {{name}} using the notes following: {{notes}}.",
    requirements=requirements_list,
    strategy=RejectionSamplingStrategy(loop_budget=5),
    user_variables={"name": name, "notes": notes},
    return_sampling_results=True,
)
if email_candidate.success:
    print(email_candidate.result)
else:
    print(email_candidate.sample_generations[0].value)

2025-10-23 11:27:45,502 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/ps "HTTP/1.1 200 OK"
2025-10-23 11:27:45,507 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
  0%|          | 0/5 [00:00<?, ?it/s]2025-10-23 11:27:47,349 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-23 11:27:47,714 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-23 11:27:47,788 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== 11:27:47-INFO ======
SUCCESS


2025-10-23 11:27:47,790 - INFO - SUCCESS
  0%|          | 0/5 [00:02<?, ?it/s]

subject: heartfelt appreciation for your invaluable assistance

dear olivia,

i hope this message finds you well. i am writing to express my sincere gratitude for the exceptional support and dedication you've shown in organizing events, advertising our speaker series, and resolving issues with snack delivery during the past few weeks.

your efforts have not gone unnoticed and have significantly contributed to the success of our lab's initiatives. we are truly fortunate to have someone as committed and capable as you on board.

thank you once again for your outstanding work and commitment. please accept this email as a token of our appreciation.

warm regards,

[your name]


## Solution

```py
<TBD>
```

<hr/>

## Lab 3: Generative Stubs

## Mellea features

In classical programming, pure (stateless) functions are a simple and powerful abstraction. A pure function takes inputs, computes outputs, and has no side effects. Generative programs can also use functions as abstraction boundaries, but in a generative program the meaning of the function can be given by an LLM instead of an interpreter or compiler. This is the idea behind a GenerativeSlot.

A GenerativeSlot is a function whose implementation is provided by an LLM. In Mellea, you define these using the `@generative` decorator. The function signature specifies the interface, and the docstring (or type annotations) guide the LLM in producing the output. Let's start with a simple example of a sentiment classifier using the Generative interface.

## Use-case illustration

Setup a classifier to detect the sentiment of the client's email, and use it to customize our answer.

## Lab guidelines

- use the `@generative` annotation to create a runnable method from a text description
- determine possible outputs with the `Literal[]` type annnotation

## Let's experiment!

In [24]:
import mellea
from mellea import generative
from typing import Literal

m = mellea.start_session()

@generative
def sentiment_classifier(text: str) -> None | Literal["positive", "negative"]:
    """Determine if the sentiment of `text` is positive or negative."""

@generative
def return_reason_detection(text: str) -> None | Literal["size", "color"]:
    """Determine if the reason for the product return described in `text` is size or color."""

sentiment = sentiment_classifier(m, text="The weather in Orlando is beautiful today!")
print(f"Detected sentiment: {sentiment}")

reason = return_reason_detection(m, text="The Tshirt is too small.")
print(f"Detected reason: {reason}")

2025-10-23 11:27:47,827 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/ps "HTTP/1.1 200 OK"
2025-10-23 11:27:47,830 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
  0%|          | 0/2 [00:00<?, ?it/s]2025-10-23 11:27:48,199 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== 11:27:48-INFO ======
SUCCESS


2025-10-23 11:27:48,203 - INFO - SUCCESS
  0%|          | 0/2 [00:00<?, ?it/s]


Detected sentiment: positive


  0%|          | 0/2 [00:00<?, ?it/s]2025-10-23 11:27:48,552 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== 11:27:48-INFO ======
SUCCESS


2025-10-23 11:27:48,555 - INFO - SUCCESS
  0%|          | 0/2 [00:00<?, ?it/s]

Detected reason: size


## Solution

```py
import mellea
from mellea import generative
from typing import Literal

m = mellea.start_session()

@generative
def sentiment_classifier(text: str) -> None | Literal["positive", "negative"]:
    """Determine if the sentiment of `text` is positive or negative."""

@generative
def return_reason_detection(text: str) -> None | Literal["size", "color"]:
    """Determine if the reason for the product return described in `text` is size or color."""

sentiment = sentiment_classifier(m, text="The weather in Orlando is beautiful today!")
print(f"Detected sentiment: {sentiment}")

reason = return_reason_detection(m, text="The Tshirt is too small.")
print(f"Detected reason: {reason}")
```

<hr/>

# Lab 4: Docling and Mellea

## Mellea features

In this lab, we will use both Docling and Mellea to extract data from a PDF.

## Use-case illustration

Use docling to read the invoice attached by the client and check for the product reference.

## Lab guidelines

- use the `RichDocument.from_document_file` to load a pdf file
- use `get_tables()` to retrieve the tables in the document
- use `m.query()` to ask a question on the table

## Let's experiment!

In [25]:
import mellea
from mellea.stdlib.docs.richdocument import RichDocument
from mellea.stdlib.docs.richdocument import Table

rd = RichDocument.from_document_file("./assets/Mellea Store - Invoice.pdf")

table1: Table = rd.get_tables()[0]
print(table1.to_markdown())

m = mellea.start_session()
ref = m.query(table1, "what is the product number ?")
print(ref)

2025-10-23 11:27:48,571 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-23 11:27:48,573 - INFO - Going to convert document batch...
2025-10-23 11:27:48,573 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 77ade624737fd5e44f3478e1fd0b64f8
2025-10-23 11:27:48,573 - INFO - Auto OCR model selected ocrmac.
2025-10-23 11:27:48,574 - INFO - Accelerator device: 'mps'
2025-10-23 11:27:49,425 - INFO - Accelerator device: 'mps'
2025-10-23 11:27:49,690 - INFO - Processing document Mellea Store - Invoice.pdf
2025-10-23 11:27:50,885 - INFO - Finished converting document Mellea Store - Invoice.pdf in 2.32 sec.
2025-10-23 11:27:50,907 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/ps "HTTP/1.1 200 OK"
2025-10-23 11:27:50,919 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"


| Item                           | Product Number   | Quantit y   | Unit Price   | Total   |
|--------------------------------|------------------|-------------|--------------|---------|
| Amanita Muscaria Tee (Black M) | #AMT3569         | 1           | $58          | $58     |
|                                |                  |             | Subtotal     | $58     |
|                                |                  |             | Tax (10%)    | $5.8    |


2025-10-23 11:27:51,737 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


The product number for Amanita Muscaria Tee (Black M) is #AMT3569.


## Solution

```py
import mellea
from mellea.stdlib.docs.richdocument import RichDocument
from mellea.stdlib.docs.richdocument import Table

rd = RichDocument.from_document_file("./assets/Mellea Store - Invoice.pdf")

table1: Table = rd.get_tables()[0]
print(table1.to_markdown())

m = mellea.start_session()
ref = m.query(table1,"what is the product number ?")
print(ref)
print(f'Product number: {ref.value.split("#")[1]}')
```

<hr/>

# Lab 5: Vision models

## Mellea features

Analyse an image by leveraging a vision model directly in Mellea

## Use-case illustration

Use a vision model to read a photo provided by the client and validate the item to be returned.

## Lab guidelines

- start a mellea session with a model vision
- load an image
- pass the image as context to the `instruct()` method

## Let's experiment!

In [26]:
import mellea
from PIL import Image

# Initiate a session with a model vision
m = mellea.start_session(model_id="granite3.2-vision")

# load image
test_img = Image.open("./assets/photo-mellea-shoe.png")

# ask a question about the 1st image
res = m.instruct("Is this a T-shirt ?", images=[test_img])
print(f"Photo 1 result: {res.value}")

# ask a question about the 2nd image
print(
    f"Photo 2 result: {m.instruct("Is this a T-shirt ?", images=[Image.open("./assets/photo-mellea-tshirt.png")])}"
)

2025-10-23 11:27:51,764 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/ps "HTTP/1.1 200 OK"
2025-10-23 11:27:51,767 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
  0%|          | 0/2 [00:00<?, ?it/s]2025-10-23 11:28:06,593 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== 11:28:06-INFO ======
SUCCESS


2025-10-23 11:28:06,600 - INFO - SUCCESS
  0%|          | 0/2 [00:14<?, ?it/s]


Photo 1 result: 
no


  0%|          | 0/2 [00:00<?, ?it/s]2025-10-23 11:28:12,179 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== 11:28:12-INFO ======
SUCCESS


2025-10-23 11:28:12,180 - INFO - SUCCESS
  0%|          | 0/2 [00:05<?, ?it/s]

Photo 2 result: 
Yes


## Solution

```py
import mellea
from PIL import Image

# Initiate a session with a model vision
m = mellea.start_session(model_id="granite3.2-vision")

# load image
test_img = Image.open("./assets/photo-mellea-shoe.png")

# ask a question about the 1st image
res = m.instruct("Is this a T-shirt ?", images=[test_img])
print(f"Photo 1 result: {res.value}")

# ask a question about the 2nd image
print(
    f"Photo 2 result: {m.instruct("Is this a T-shirt ?", images=[Image.open("./assets/photo-mellea-tshirt.png")])}"
)
```

<hr/>